<a href="https://colab.research.google.com/github/vbbot/test-repo/blob/master/Copy_of_linear_regression_prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install ftx

In [ ]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
import ftx
import matplotlib.pyplot as plt
from dateutil import parser
from datetime import timedelta, date
import datetime as dt
plt.style.use('fivethirtyeight')

In [ ]:
client = ftx.FtxClient(api_key="", api_secret="")
klines = client.get_historical_data("BTC-PERP", 86400, 2000)
df = pd.DataFrame(klines)
df 

for x in range(30):
  last_date = df.iloc[-1]['startTime']
  a = parser.parse(str(last_date))
  a = a + timedelta(days=1)
  str(a)

  new_row = df.iloc[-1]
  df_new_row = pd.DataFrame(new_row)
  df_new_row.iloc[0] = a
  df_t = df_new_row.transpose()
  df = pd.concat([df, df_t], axis=0)

df.reset_index(drop=True, inplace=True)
df

In [ ]:
# Visualising the closing price history
plt.figure(figsize=(16,8))
plt.title('Close Price History')
plt.plot(df['close'])
plt.xlabel('Date', fontsize=18)
plt.ylabel('Close Price USD', fontsize=18)
plt.show()

In [ ]:
# prediction x days into the future
projection = 90

df['Prediction'] = df[['close']].shift(-projection)
df.tail

In [ ]:
# Create the independent dataset
x = np.array(df['close'])

# remove last 14 rows
x = x[:-projection]
x

In [ ]:
# Create dependent dataset
y = df['Prediction'].values
y = y[:-projection]
y

In [ ]:
# Split data into 85% training and 15% testing dataset
x_train, x_test, y_train, y_test = train_test_split(x,y, test_size=.15)

# x_train = x_train.reshape(1,-1)
# y_train = y_train.reshape(1,-1)
# Create and train the model

x_train, y_train = np.array(x_train), np.array(y_train)
x_train = x_train.reshape(-1,1)
x_test = x_test.reshape(-1,1)
# y_train = y_train.reshape(-1,1)
# x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], 1))

lin_reg = LinearRegression()
lin_reg.fit(x_train, y_train)

linreg_confidence = lin_reg.score(x_test,y_test)
linreg_confidence

In [ ]:
x_projection = np.array(df['close'])[-projection:]
x_projection = x_projection.reshape(-1,1)
x_projection

In [ ]:
#Print the linear reg predictions for next 14 days
lin_reg_prediction = lin_reg.predict(x_projection)
lin_reg_prediction




In [ ]:
close_values =  df['close'].values
closes = close_values
closes = closes[:-projection]
combined = np.concatenate((closes, lin_reg_prediction), axis=0)
print(close_values[len(close_values) - 100:])
print(combined[len(combined) - 100:])

In [ ]:
data = df.filter(['close'])
valid = data[len(closes):]
valid['Predictions'] = lin_reg_prediction

# Visualising the closing price history
plt.figure(figsize=(16,8))
plt.title('Close Price History')
plt.plot(df['close'], linewidth=2)
plt.plot(combined, linewidth=2, color='red')
# plt.plot(combined.iloc[:15,:])
# plt.plot(valid[['close', 'Predictions']], linewidth=2)
plt.xlabel('Date', fontsize=18)
plt.ylabel('Close Price USD', fontsize=18)
plt.show()